In [1]:
import pandas as pd

https://ale.farama.org/environments/frogger/
\n
https://ale.farama.org/environments/

In [4]:
import gymnasium
import os
import ale_py

In [5]:
gymnasium.make("ALE/Frogger-v5")

<OrderEnforcing<PassiveEnvChecker<AtariEnv<ALE/Frogger-v5>>>>